In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image

In [ ]:
class CustomDataset(Dataset):
    # the dictionary will be: -
    #     key -> image path
    #     value -> regression values
    
    def __init__(self, image_dict, transform=None):
        self.image_dict = image_dict
        self.image_paths = list(image_dict.keys()) # add keys in array
        self.transform = transform
        
    def __getitem__(self, index):
        image_path = self.image_paths[index]
        regression_values = self.image_dict[image_path]
        
        image = Image.open(image_path).convert('L')
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, regression_values
    
    def __len__(self):
        return len(self.image_paths)

In [ ]:
import pandas as pd
import os

In [ ]:
data = pd.read_csv('C:\\Users\\iqmal_pc\\Desktop\\fyp_experiment\\angles_ap_labelled.csv')
data

In [ ]:
# get train image list and its path
image_path = 'C:\\Users\\iqmal_pc\\Desktop\\fyp_experiment\\sobel_img_data\\train\\'
images_jpg = data['image_train'].values.tolist()

# append filepath
for i in range(len(images_jpg)):
    images_jpg[i] = image_path + images_jpg[i]


In [ ]:
image_dict = {}
for i in range(len(images_jpg)):
    image_dict[images_jpg[i]] = torch.tensor([data.loc[i, 'PT'], data.loc[i, 'MT'], data.loc[i, 'TL/L']], dtype=torch.float32)
    

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [ ]:
# neural network arch

class My_Regression_Model(nn.Module):
    def __init__(self, num_channel):
        super().__init__()
        
        # initialize 1st conv=>relu=>pool
        self.conv1 = nn.Conv2d(in_channels=num_channel, out_channels=50, kernel_size=(100,100))
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        
        # initialize 2nd conv=>relu=>pool
        self.conv2 = nn.Conv2d(in_channels=50, out_channels=1, kernel_size=(15,15))
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        
        # initialize 1st fc=>relu
        self.fc1 = nn.Linear(in_features=1*32*32, out_features=300)
        self.relu3 = nn.ReLU()
        
        # final fc
        self.fc2 = nn.Linear(in_features=300, out_features=3)
        
    def forward(self, x): # c, h, w
        x = self.conv1(x) # in -> (1, 255, 255) | out -> (50, 156, 156)
        x = self.relu1(x) 
        x = self.maxpool1(x) # in -> (50, 156, 156) | out -> (50, 78, 78)
        
        x = self.conv2(x) # in -> (50, 78, 78) | out -> (1, 64, 64)
        x = self.relu2(x) 
        x = self.maxpool2(x) # in -> (1, 64, 64) | out -> (1, 32, 32)
        
        # flatten
        x = torch.flatten(x,1)
        
        # fc1
        x = self.fc1(x)
        x = self.relu3(x)
        
        # fc2
        output = self.fc2(x)
        return output

In [ ]:
# Define the input channels
input_channels = 1  # Assuming L images

# Define the transformation to apply to the images
transform = transforms.Compose([
    transforms.Resize((255, 255)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.type(torch.float32))
])

In [ ]:
# Create an instance of the custom dataset
dataset = CustomDataset(image_dict, transform=transform)

In [ ]:
image_num = 146
daImg, regress_val = dataset.__getitem__(image_num)
daImg = daImg.squeeze()
print(daImg)
print('Img ' + str(image_num), daImg.shape)
import matplotlib.pyplot as plt
plt.imshow(daImg, cmap='gray')
print('Img ' + str(image_num) + ' regress_val', regress_val)

In [ ]:
data.loc[image_num]

In [ ]:
# Create a data loader for batching and shuffling the data
data_loader = DataLoader(dataset, batch_size=37, shuffle=True)

In [ ]:
# Create an instance of the regression model
model = My_Regression_Model(input_channels)

In [ ]:
# Move the model to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# import matplotlib.pyplot as plt

# for idx, (batch_data, batch_targets) in enumerate(data_loader):
#     if idx == 0: # only for 1st batch out of 13 batches
#         for i in range(batch_data.shape[0]):
#             print('image ' + str(i))
#             print(batch_data[i])
#             print('SHAPE BATCH DATA', batch_data[i].shape)
#             print('DATATYPE BATCH DATA', batch_data.dtype)
    
#             # convert to image
#             image_arr = batch_data[i].numpy()
#             image_arr = image_arr.squeeze()
#             image_arr = image_arr.astype('uint8')
#             plt.imshow(image_arr, cmap='gray')
#             plt.show()
    
#             print(batch_targets[i])
#             print('SHAPE BATCH TARGET', batch_targets[i].shape)
#             print('DATATYPE BATCH TARGET', batch_targets[i].dtype)
#             print('----------------------------------------------------------------')

In [ ]:
# Define the loss function
criterion = nn.MSELoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
number_of_batches = len(data_loader)
num_epochs = 200
for epoch in range(num_epochs):
    # training phase
    batch_num = 0
    total_loss_in_epoch = 0
    model.train()
    for images, targets in data_loader:
        images = images.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        batch_num += 1
        total_loss_in_epoch += loss.item()
        # print(f"Batch: {batch_num}, Loss: {loss.item()}")
        
    # validation phase
    model.eval() # to stop all parameter adjustment
    val_loss = 0.0
    with torch.no_grad():
        for

    # Print the loss for every epoch
    print(f"Epoch: {epoch+1}, Loss: {total_loss_in_epoch/number_of_batches}")
    print('--------------------------------------------------')

In [ ]:
# Make predictions on new data
new_image = Image.open('path_to_new_image.jpg').convert('L')
new_image_tensor = transform(new_image).unsqueeze(0).to(device)
predicted_values = model(new_image_tensor).squeeze().tolist()
print("Predicted values:", predicted_values)